In [ ]:
from Functions1 import LoadData
from Functions1 import TestStrategy
from Functions1 import DynamicBacktest
import os
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import random
import matplotlib.pyplot as plt

### Carga de datos

In [ ]:
current_directory = os.getcwd()
path = os.path.join(current_directory, 'Stocks_financials')

loader = LoadData(path)
prices, financials = loader.Load(start_date='2005-09-01', end_date=None)

### Escenario unico

In [ ]:
offensive_tickers = ["MSFT", "AAPL", "GOOGL", 'V', 'NFLX', 'SO', 'DUK', 'VZ', 'T', 'CVX', 'VMC', 'NVDA', 'GE', 'KO', 'JNJ', 'HD', 'AMD', 'ADM', 'META', 'TSLA', 'XOM']   
defensive_tickers = ['COST', 'DUK', 'JNJ', 'KO', 'LMT']      
canary_tickers = ['MSFT', 'V', 'NFLX', 'SO', 'DUK', 'PLD', 'VZ', 'T', 'CVX', 'VMC', 'NVDA', 'GE',
                      'KO', 'JNJ', 'GS', 'HD', 'AMD', 'ADM', 'META', 'GOOGL', 'TSLA', 'JPM', 'XOM', 'AAPL']    

In [ ]:
strategy = TestStrategy(prices, financials, offensive_tickers, defensive_tickers, canary_tickers)
chosen_universe = strategy.run_strategy(start_date='2005-09-01')
chosen_universe.head(8)

In [ ]:
backtest = DynamicBacktest(chosen_universe, prices, 1000000)
port_values = backtest.get_portfolio_values()
port_values

In [ ]:
backtest.plot_strategies()

In [ ]:
backtest.evaluate_portfolios().T

-------

### Simulaciones del metodo con Portafolios aleatorios

In [ ]:
offensive_tickers = ['AAPL', 'ADM', 'ADSK', 'AMD', 'CAT', 'CRM', 'CVX', 'DE', 'DOW', 'GE', 'GOOGL', 'HD', 'HON',
                     'MMM', 'MSFT', 'NFLX', 'NVDA', 'QCOM', 'RTX', 'TGT', 'TRYIY', 'TSLA', 'TXN', 'V', 'VMC', 'XOM']

defensive_tickers = ['CL', 'COST', 'DUK', 'JNJ', 'KO', 'LMT', 'MCD', 'MDLZ', 'MRK', 'PG', 'SO',
 'SRE', 'T', 'VZ', 'WMT', 'ZBH'] 

canary_tickers = [
    # Tecnología
    'AAPL', 'MSFT', 'NVDA',
    # Finanzas
    'JPM', 'GS',
    # Energía
    'XOM', 'CVX',
    # Consumo Discrecional
    'TSLA', 'HD',
    # Consumo Básico
    'PG', 'KO',
    # Salud
    'JNJ', 'ABBV',
    # Industriales
    'CAT', 'HON', 'MMM',
    # Telecomunicaciones
    'VZ', 'T',
    # Bienes Raíces
    'PLD',
    # Materiales
    'VMC',
    # Utilidades
    'DUK'
]

In [ ]:
n_portfolios = 100

def get_random_combinations(tickers, n, sample_size):
    try:
        return random.sample(list(itertools.combinations(tickers, n)), sample_size)
    except ValueError as e:
        print(f"Error while generating combinations: {e}")
        return []

offensive_combinations = get_random_combinations(offensive_tickers, 16, n_portfolios)
defensive_combinations = get_random_combinations(defensive_tickers, 5, n_portfolios)

results = []

for offensive_combo, defensive_combo in zip(offensive_combinations, defensive_combinations):
    

    try:
        strategy = TestStrategy(prices, financials, list(offensive_combo), list(defensive_combo), canary_tickers)
        
        chosen_universe = strategy.run_strategy(start_date='2005-09-01')
        
        backtest = DynamicBacktest(chosen_universe, prices, 1000000)
        port_values = backtest.get_portfolio_values()
        evaluation = backtest.evaluate_portfolios().T
        
        results.append({
            'Offensive Combination': offensive_combo,
            'Defensive Combination': defensive_combo,
            'Evaluation': evaluation
        })
        
    except Exception as e:
        print(f"Error with offensive {offensive_combo} and defensive {defensive_combo}: {str(e)}")

final_results = pd.DataFrame(results)

In [ ]:
all_evaluations = pd.concat([result['Evaluation'] for result in results], keys=range(len(results)))

average_evaluation = all_evaluations.groupby(level=1).mean()

In [ ]:
metric_summary = pd.DataFrame({
    'Offensive Combination': [result['Offensive Combination'] for result in results],
    'Defensive Combination': [result['Defensive Combination'] for result in results],
    'Sharpe Ratio': [result['Evaluation'].loc['Sharpe Ratio', 'Sharpe'] for result in results],
    'CAGR': [result['Evaluation'].loc['CAGR', 'Sharpe'] for result in results],
    'Max Drawdown': [result['Evaluation'].loc['Max Drawdown', 'Sharpe'] for result in results]
})

best_sharpe_portfolio = metric_summary.loc[metric_summary['Sharpe Ratio'].idxmax()]
best_cagr_portfolio = metric_summary.loc[metric_summary['CAGR'].idxmax()]
best_drawdown_portfolio = metric_summary.loc[metric_summary['Max Drawdown'].idxmin()]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
average_evaluation['Sharpe'].plot(ax=ax, label='Promedio de todas las combinaciones', linestyle='--', color='blue')
best_sharpe_portfolio['Sharpe Ratio'].plot(ax=ax, label='Mejor Portafolio (Sharpe Ratio)', linestyle='-', color='green')
plt.title('Comparación de Rendimiento Basado en Sharpe Ratio')
plt.ylabel('Sharpe Ratio')
plt.xlabel('Fecha')
plt.legend()
plt.grid(True)
plt.show()

--------

In [ ]:

ratios_to_check = [
    'Return on Common Equity',
    'Operating Margin',
    'Cash Flow per Share',
    'Current Ratio (x)',
    'Total Debt/Equity (%)',
    'Total Shares Outstanding  (M)',
    'Price/Earnings'
]

tickers_with_all_ratios = []

for ticker, df in financials.items():
    if all(ratio in df['Financial Ratio'].values for ratio in ratios_to_check):
        tickers_with_all_ratios.append(ticker)

tickers_with_all_ratios

In [ ]:
all_tickers = [
    'AAPL', 'ABBV', 'ADM', 'ADSK', 'AFL', 'AMD', 'CAT', 'CL', 'COST', 'CRM', 'CVX',
    'DE', 'DOW', 'DUK', 'GE', 'GOOGL', 'GS', 'HD', 'HON', 'JNJ', 'JPM', 'KO', 'LMT', 
    'MCD', 'MDLZ', 'META', 'MMM', 'MRK', 'MSFT', 'NFLX', 'NVDA', 'PG', 'PLD', 'QCOM', 
    'RTX', 'SO', 'SRE', 'T', 'TGT', 'TRYIY', 'TSLA', 'TXN', 'V', 'VMC', 'VZ', 'WMT', 
    'XOM', 'ZBH'
]

tickers_without_all_ratios = [ticker for ticker in all_tickers if ticker not in tickers_with_all_ratios]
tickers_without_all_ratios

In [ ]:
tickers_to_check = [
    'ABBV', 'CL', 'COST', 'DUK', 'JNJ', 'KO', 'LMT', 'MCD', 'MDLZ', 'MRK', 
    'PG', 'SO', 'SRE', 'T', 'VZ', 'WMT', 'ZBH'
]

tickers_with_initial_date = []

for ticker in tickers_to_check:
    first_date = pd.to_datetime(prices[ticker].index[0])
    
    if first_date == pd.to_datetime('2005-09-01'):
        tickers_with_initial_date.append(ticker)

tickers_with_initial_date

In [ ]:
missing_tickers = [ticker for ticker in canary_tickers if ticker not in all_tickers]

missing_tickers